In [27]:
import pandas as pd
import numpy as np

In [55]:
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic
from statsmodels.stats.proportion import proportion_confint,samplesize_confint_proportion
import plotly.express as px
from scipy.stats import norm
from math import ceil

# Quiz 1

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на миллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

In [4]:
water = pd.read_csv('water.txt', sep= '\t')

In [6]:
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [5]:
water.describe()

,mortality,hardness
count,61.000000,61.000000
mean,1524.147541,47.180328
std,187.668754,38.093966
min,1096.000000,5.000000
25%,1379.000000,14.000000
50%,1555.000000,39.000000
75%,1668.000000,75.000000
max,1987.000000,138.000000


Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы не думать всё время о том, правильно ли вычисляется в вашем случае std(), можно всегда использовать std(ddof=1) (ddof — difference in degrees of freedom), тогда нормировка всегда будет на n-1.

In [29]:
std_mort = water.std(ddof=1).mortality/np.sqrt(water.shape[0])
mean_mort = water.mortality.mean()

In [30]:
_tconfint_generic(mean_mort, std_mort, water.shape[0] - 1, 0.05, 'two-sided')

(1476.0833413552848, 1572.2117406119285)

На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [33]:
south = water[water.location == 'South']
std_mort_s = south.std(ddof=1).mortality/np.sqrt(south.shape[0])
mean_mort_s = south.mortality.mean()

In [34]:
_tconfint_generic(mean_mort_s, std_mort_s, south.shape[0] - 1, 0.05, 'two-sided')

(1320.1517462936238, 1433.463638321761)

На тех же данных постройте 95% доверительный интервал для средней годовой смертности по всем северным городам. Пересекается ли этот интервал с предыдущим? Как вы думаете, какой из этого можно сделать вывод?

In [35]:
north = water[water.location == 'North']
std_mort_n = north.std(ddof=1).mortality/np.sqrt(north.shape[0])

mean_mort_n = north.mortality.mean()

In [36]:
_tconfint_generic(mean_mort_n, std_mort_n, north.shape[0] - 1, 0.05, 'two-sided')

(1586.5605251961385, 1680.6394748038613)

Интервалы не пересекаются; видимо, средняя смертность на севере и на юге существенно разная

Пересекаются ли 95% доверительные интервалы для средней жёсткости воды в северных и южных городах?

In [37]:
std_hard_n = north.std(ddof=1).hardness/np.sqrt(north.shape[0])
mean_hard_n = north.hardness.mean()

In [39]:
std_hard_s = south.std(ddof=1).hardness/np.sqrt(south.shape[0])
mean_hard_s = south.hardness.mean()

In [40]:
_tconfint_generic(mean_hard_n, std_hard_n, north.shape[0] - 1, 0.05, 'two-sided'),\
_tconfint_generic(mean_hard_s, std_hard_s, south.shape[0] - 1, 0.05, 'two-sided')

((21.42248728572426, 39.37751271427574),
 (53.467198692036106, 86.07126284642544))

Не пересекаются

Определение нужного объема выборки

np.ceil((stats.norm.ppf(1-0.05/2) / 0.1)**2)

# Quiz 2

Большая часть млекопитающих неспособны во взрослом возрасте переваривать лактозу, содержащуюся в молоке. У людей за расщепление лактозы отвечает фермент лактаза, кодируемый геном LCT. У людей с вариантом 13910T этого гена лактаза продолжает функционировать на протяжении всей жизни. Распределение этого варианта гена сильно варьируется в различных генетических популяциях.

Из 50 исследованных представителей народа майя вариант 13910T был обнаружен у одного. Постройте нормальный 95% доверительный интервал для доли носителей варианта 13910T в популяции майя. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [75]:
proportion_confint(1,  50, method = 'normal', alpha = 0.05)

(0.0, 0.05880530708179099)

В условиях предыдущей задачи постройте 95% доверительный интервал Уилсона для доли носителей варианта 13910T в популяции майя. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [51]:
proportion_confint(1,  50, method = 'wilson', alpha = 0.05)

(0.003539259271646236, 0.10495443589637815)

Пусть в популяции майя действительно 2% носителей варианта 13910T, как в выборке, которую мы исследовали. Какой объём выборки нужен, чтобы с помощью нормального интервала оценить долю носителей гена 13910T с точностью \pm0.01±0.01 на уровне доверия 95%?

In [58]:

ceil(samplesize_confint_proportion(1.0/50, 0.01, method='normal'))

753

Постройте график зависимости объёма выборки, необходимого для оценки для доли носителей гена 13910T с точностью \pm0.01±0.01 на уровне доверия 95%, от неизвестного параметра pp. Посмотрите, при каком значении pp нужно больше всего испытуемых. Как вы думаете, насколько вероятно, что выборка, которую мы анализируем, взята из случайной величины с этим значением параметра?

Как бы вы не ответили на последний вопрос, рассмотреть объём выборки, необходимый при таком pp, всё равно полезно — это даёт максимально пессимистичную оценку необходимого объёма выборки.

Какой объём выборки нужен в худшем случае, чтобы с помощью нормального интервала оценить долю носителей гена 13910T с точностью \pm0.01±0.01 на уровне доверия 95%?

In [60]:
n_size = []
for p in range(0,100):
    n_size.append(ceil(samplesize_confint_proportion(p/100, 0.01, method='normal')))

In [74]:
fig = px.line( x=list(range(0,100)), y=n_size)
fig.show()

In [73]:
max(n_size)

9604